In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports

url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fnationalaccounts%2fbalanceofpayments%2fdatasets%2fuktradecountrybycommodityimports%2fcurrent/countrybycommodityimports.zip"

In [3]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Import")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,2018JAN,2018FEB,2018MAR,2018APR,2018MAY,2018JUN,2018JUL,...,2021JAN,2021FEB,2021MAR,2021APR,2021MAY,2021JUN,2021JUL,2021AUG,2021SEP,2021OCT
0,0 Food & live animals,AD Andorra,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.07,0.00,0.07,0.00,0.00,0.00,0.00,0.00
1,0 Food & live animals,AE United Arab Emirates,IM Imports,0.71,0.71,8.59,1.43,0.85,0.93,0.79,...,2.22,1.20,5.41,1.54,1.18,1.32,1.52,1.76,1.33,1.03
2,0 Food & live animals,AF Afghanistan,IM Imports,0.06,0.00,0.00,0.00,0.02,0.12,0.00,...,0.11,0.24,0.10,0.12,0.02,0.00,0.03,0.01,0.00,0.00
3,0 Food & live animals,AG Antigua and Barbuda,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.02,0.00,0.04,0.00,0.00,0.05,0.00
4,0 Food & live animals,AI Anguilla,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "imports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.imports = df.imports*1000000

In [7]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"imports": "sum"})

In [8]:
topcntry = test.sort_values(by = "imports", ascending = False).iloc[0:23].index.to_list()

In [9]:
df = df[df.COUNTRY.isin(topcntry)]

In [10]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [11]:
grp = df.groupby(["COUNTRY"])

In [12]:
def ex_metal_total(df):
        
    if not (df[df["COMMODITY"] == "9 Unspecified goods"].empty):
                
        foo = pd.DataFrame([])

        fooall = df[df["COMMODITY"] == "Total"].set_index(["time"])["imports"] 
        
        foobar = df[df["COMMODITY"] == "9 Unspecified goods"].set_index(["time"])["imports"]

        foo["imports"] = fooall - foobar
    # Grab the total and then sbutract off the unspecified goods
    
        foo["COUNTRY"] = df.COUNTRY.unique()[0]
    
        #print(df.COUNTRY.unique())
    
        foo["COMMODITY"] = "Total (ex. metals)"
        
        foo.reset_index(inplace = True)
        
        foo[df.columns]
        
        df = df.append(foo)
    
    return df

In [13]:
df = grp.apply(ex_metal_total)

In [14]:
df.drop(["COUNTRY"], axis = 1, inplace = True)

df.reset_index(inplace = True)

df.drop(["level_1"], axis = 1, inplace = True)

In [15]:
file = ".\\data"+ "\\UK-imports-1997-2017.parquet"

foo = pq.read_table(file).to_pandas()

In [17]:
df

,COUNTRY,COMMODITY,time,imports
0,Belgium,0 Food & live animals,2018-01-01,1.734300e+08
1,Belgium,00 Live animals,2018-01-01,6.000000e+04
2,Belgium,01 Meat & meat preparations,2018-01-01,1.547000e+07
3,Belgium,02 Dairy products & eggs,2018-01-01,1.761000e+07
4,Belgium,03 Fish & shellfish,2018-01-01,1.200000e+06
...,...,...,...,...
133303,Whole world,Total (ex. metals),2021-06-01,3.856800e+10
133304,Whole world,Total (ex. metals),2021-07-01,3.898800e+10
133305,Whole world,Total (ex. metals),2021-08-01,3.606200e+10
133306,Whole world,Total (ex. metals),2021-09-01,4.264700e+10


In [18]:
df = df.append(foo) #add the historical data

In [19]:
df

,COUNTRY,COMMODITY,time,imports
0,Belgium,0 Food & live animals,2018-01-01,1.734300e+08
1,Belgium,00 Live animals,2018-01-01,6.000000e+04
2,Belgium,01 Meat & meat preparations,2018-01-01,1.547000e+07
3,Belgium,02 Dairy products & eggs,2018-01-01,1.761000e+07
4,Belgium,03 Fish & shellfish,2018-01-01,1.200000e+06
...,...,...,...,...
730291,Whole world,Total (ex. metals),2017-08-01,3.862600e+10
730292,Whole world,Total (ex. metals),2017-09-01,4.183200e+10
730293,Whole world,Total (ex. metals),2017-10-01,4.127100e+10
730294,Whole world,Total (ex. metals),2017-11-01,4.213700e+10


In [20]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [21]:
df.sort_index(inplace = True)

df.tail()

imports
COUNTRY     COMMODITY          time                    
Whole world Total (ex. metals) 2021-06-01  3.856800e+10
                               2021-07-01  3.898800e+10
                               2021-08-01  3.606200e+10
                               2021-09-01  4.264700e+10
                               2021-10-01  4.246100e+10

In [22]:
df.head()

imports
COUNTRY COMMODITY             time                  
Belgium 0 Food & live animals 1997-01-01  44370000.0
                              1997-02-01  44480000.0
                              1997-03-01  43920000.0
                              1997-04-01  59350000.0
                              1997-05-01  57810000.0

In [23]:
out_file = ".\\data"+ "\\UK-imports.parquet"

pq.write_table(pa.Table.from_pandas(df.sort_index()), out_file)